In [15]:
import json
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os
import openai

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

In [2]:
load_dotenv(find_dotenv(), override=True)

True

In [3]:
with open('reviews_output.json', 'r') as f:
    data = json.load(f)

In [11]:
model_version = "gpt-4-1106-preview"
chat_llm=ChatOpenAI(temperature=0, model_name=model_version)

In [46]:
def request_chat(messages):
    try:
        response = openai.chat.completions.create(
            model='gpt-4-1106-preview',
            messages=messages,
            response_format={ "type": "json_object" }
        )
    
        return response
    except openai.APIError as e:
        # Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
    
    except openai.AuthenticationError as e:
        # Handle Authentication error here, e.g. invalid API key
        print(f"OpenAI API returned an Authentication Error: {e}")
    
    except openai.APIConnectionError as e:
        # Handle connection error here
        print(f"Failed to connect to OpenAI API: {e}")
    
    
    except Exception as e:
        # Handles all other exceptions
        print(e)

In [51]:
def request(reviews_data):
    system_prompt = "You are a helpful assistant that finds reviews that are relevant to the restaurant."
    human_prompt = """
Given a restaurant's reviews and their IDs below, find 10 positive reviews and 10 negative reviews regarding each of the topics listed below:
1. Service
2. Hygiene and environment
3. Food quality and taste
4. Location and transportation
5. Price
When returning the results, put the reviews that have strong opinions in the front and make sure to include the IDs of the reviews with the json format below:
{
    Category: { 
        "Positive": {'ID: review'}
        "Negative": {'ID: review'}
    }
}
========
EXAMPLE ANSWER:
{
"Services": {
    "Positive": {
        3: "The service is superb!",
        4: "The waiters are very well trained."
    },
    "Negative": {
        10: "The owner's attitude is the worst!",
        15: "The waiters need some training."
    }
},
"Hygiene and environment": {
    "Positive": {
        20: "The environment is superb!",
        24: "The restaurant is very well cleaned."
    },
    "Negative": {
        30: "The hygiene is the worst!",
        36: "Some of the plates are clearly not washed."
    }
},
"Food quality and taste": {
    "Positive": {
        47: "The food here is exceptional!"
    },
    "Negative": {
        49: "The taste is awful!"
    }
},
"Location and transportation": {
    "Positive": {
        51: "There is good parking here."
    },
    "Negative": {
        52: "This place is so hard to get to."
    }
},
"Price": {
    "Positive": {
        1: "The food is superb for this price!"
    },
    "Negative": {
        11: "Food here is so expensive!"
    }
} 

========
Reveiws:
"""
    for x in reviews_data:
        human_prompt += str(x['id']) + ": " + x['text'] + '\n'
    human_prompt += """========
ANSWER:
"""
    messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": human_prompt}
    ]
    result = request_chat(messages)
    return result

res = request(data)
print(res)

ChatCompletion(id='chatcmpl-8MLCiqIyNPj99gaPM163E9wztb5UV', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{\n    "Service": {\n        "Positive": {\n             "23": "Our waitress was very attentive and checked in on us repeatedly.",\n            "66": "Great waiter. Fast service.",\n            "160": "Service was excellent. Lauryn was the best.",\n            "166": "The servers all had masks on.",\n            "170": "Service was excellent. Lauryn was the best."\n        },\n        "Negative": {\n            "1": "The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually long waiting.",\n            "5": "Service is poor. My mother and I waited over an hour to get our entrees and our server never apologized for the wait.",\n            "6": "Place was full and service was poor.",\n            "10": "The service is deplorable.",\n            "13": "After waiting over an hour for a table our waitre

In [61]:
result_json = json.loads(res.choices[0].message.content)

In [63]:
with open("results.json", "w") as f:
    json.dump(result_json, f)

In [50]:
def create_service_prompt(data):
    human_prompt = """
Given a restaurant's reviews and their IDs below, find 10 positive reviews and 10 negative reviews paraticularly regarding the services of the restaurant.
When returning the results, put the reviews that have strong opinions in the front and make sure to include the IDs of the reviews with the format below:
ID: review
========
EXAMPLE RESPONSE:
Positive reviews:
3: The service is superb!
4: The waiters are very well trained.

Negative reviews:
10: The owner's attitude is the worst!
15: The waiters need some training.

========
Reveiws:
{contents}
========
Again, return reviews that talks about the services of the restaurant.
Answer:
"""
    system_prompt = """You are a helpful assistant that finds reviews that are relevant to the restaurant's services.
"""

    reviews_str = ""
    for x in data:
        reviews_str += str(x['id']) + ": " + x['text'] + '\n'
        
    messages = [
        SystemMessagePromptTemplate.from_template(system_prompt),
        HumanMessagePromptTemplate.from_template(human_prompt),
    ]

    template = ChatPromptTemplate.from_messages(messages)
    formatted_messages = template.format_prompt(contents=reviews_str).to_messages()
    return formatted_messages
prompt = create_service_prompt(data)
print(prompt)

[SystemMessage(content="You are a helpful assistant that finds reviews that are relevant to the restaurant's services.\n"), HumanMessage(content='\nGiven a restaurant\'s reviews and their IDs below, find 10 positive reviews and 10 negative reviews regarding each of the topics listed below:\n1. Service\n2. Hygiene and environment\n3. Food quality and taste\n4. Location and transportation\n5. Price\nWhen returning the results, put the reviews that have strong opinions in the front and make sure to include the IDs of the reviews with the format below:\nID: review\n========\nEXAMPLE RESPONSE:\nPositive reviews:\n3: The service is superb!\n4: The waiters are very well trained.\n\nNegative reviews:\n10: The owner\'s attitude is the worst!\n15: The waiters need some training.\n\n========\nReveiws:\n1: If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it\'s other locations

In [21]:
id2review_dict = {}
for x in data:
    id2review_dict[x['id']] = x['text']

In [98]:
results = chat_llm(prompt)

In [99]:
print(results.content)

Positive reviews:
23: I was very impressed with Turning Point! My roommate had raved about it for so long, I was glad the North Wales location had finally opened so I could try it for myself. The service was great, our waitress was very attentive and checked in on us repeatedly.

16: Went here over the weekend, got my sugar rush on with the salted caramel waffles. Awesome!!! Staff was happy and helpful, noticed that kids got a little cup of fruitloops if they were waiting for a table. That's a nice touch for families.

17: This place is delicious. I like how to food is a twist on the classics. The manager is very hands on and really seems to really know the business. I really needed my check and my waiter was tending to another table, the manager noticed and got me my check and packed my leftovers asap.

26: My family loves to meet at Turning Point in Montgomeryville, PA for brunch. The restaurant is very bright and inviting. The manager and the whole staff make you feel very welcome.


In [101]:
print(id2review_dict[24])

Excellent food but slow, slow, slow. Staff needs training. I don't know if it is worth it to return. Kudos to the chef


In [8]:
def create_prompt(data):
    human_prompt = """
Given a restaurant's reviews and their IDs below, find 10 positive reviews and 10 negative reviews regarding each of the topics listed below:
1. Service
2. Hygiene and environment
3. Food quality and taste
4. Location and transportation
5. Price
When returning the results, put the reviews that have strong opinions in the front and make sure to include the IDs of the reviews with the format below:
ID: review
========
EXAMPLE RESPONSE:
Services:
Positive reviews:
3: The service is superb!
4: The waiters are very well trained.

Negative reviews:
10: The owner's attitude is the worst!
15: The waiters need some training.

Hygiene and environment:
Positive reviews:
20: The environment is superb!
24: The restaurant is very well cleaned.

Negative reviews:
30: The hygiene is the worst!
36: Some of the plates are clearly not washed.

Food quality and taste:
Positive reviews:
47: The food here is exceptional!

Negative reviews:
49: The taste is awful!

Location and transportation:
Positive reviews:
51: There is good parking here.

Negative reviews:
52: This place is so hard to get to.

Price: 
Positive reviews:
1: The food is superb for this price!

Negative reviews:
11: Food here is so expensive!

========
Reveiws:
{contents}
========
Answer:
"""
    system_prompt = """You are a helpful assistant that finds reviews that are relevant to the restaurant's several topics.
"""

    reviews_str = ""
    for x in data:
        reviews_str += str(x['id']) + ": " + x['text'] + '\n'
        
    messages = [
        SystemMessagePromptTemplate.from_template(system_prompt),
        HumanMessagePromptTemplate.from_template(human_prompt),
    ]

    template = ChatPromptTemplate.from_messages(messages)
    formatted_messages = template.format_prompt(contents=reviews_str).to_messages()
    return formatted_messages
prompt = create_prompt(data)
print(prompt)

[SystemMessage(content="You are a helpful assistant that finds reviews that are relevant to the restaurant's several topics.\n"), HumanMessage(content='\nGiven a restaurant\'s reviews and their IDs below, find 10 positive reviews and 10 negative reviews regarding each of the topics listed below:\n1. Service\n2. Hygiene and environment\n3. Food quality and taste\n4. Location and transportation\n5. Price\nWhen returning the results, put the reviews that have strong opinions in the front and make sure to include the IDs of the reviews with the format below:\nID: review\n========\nEXAMPLE RESPONSE:\nServices:\nPositive reviews:\n3: The service is superb!\n4: The waiters are very well trained.\n\nNegative reviews:\n10: The owner\'s attitude is the worst!\n15: The waiters need some training.\n\nHygiene and environment:\nPositive reviews:\n20: The environment is superb!\n24: The restaurant is very well cleaned.\n\nNegative reviews:\n30: The hygiene is the worst!\n36: Some of the plates are cl

In [12]:
results = chat_llm(prompt)

In [13]:
print(results.content)

Services:
Positive reviews:
37: Love the Kona French press here! Takes me back to Hawaii! Had the Big Easy Benedict and it was delicious! My wife had the Wilbur skillet and another great dish. The server couldn't have been more attentive and friendly. Cant wait to get back!
26: My family loves to meet at Turning Point in Montgomeryville, PA for brunch. The restaurant is very bright and inviting. The manager and the whole staff make you feel very welcome. The food is so fresh and delicious, and tends to be on the healthy side. I would highly recommend this restaurant to anyone.
3: The place is cute and the staff was very friendly. Nice menu. Good for brunch. We had lunch and were seated right away. I really enjoyed the avocado toast and side of bacon. Nice to have another brunch place nearby.
8: I went on a Thursday morning for breakfast. Staff was attentive and the omelette was excellent.
17: This place is delicious. I like how to food is a twist on the classics. I love the vanilla cha

In [106]:
with open('services.txt', 'w') as f:
    f.write(results.content)
with open('hygiene.txt', 'w') as f:
    f.write(hygiene_results.content)

In [ ]:
#price, quality, transportation